# Cats and Dogs Toy Example

This notebook will train, quantize, and synthesis the Cats and Dogs example. We want to use this example to show that a high test set accuracy does not guarantee a high accuracy on the board.

### Import the necessary modules

In [91]:
import importlib

import os
from matplotlib.image import imread
import numpy as np
import matplotlib.pyplot as plt
import time
import logging
from collections import OrderedDict

try:
    import tensorboard  # pylint: disable=import-error
    import tensorflow  # pylint: disable=import-error
    tensorflow.io.gfile = tensorboard.compat.tensorflow_stub.io.gfile
except (ModuleNotFoundError, AttributeError):
    pass

import torch
from torch import nn
import torchnet.meter as tnt
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import nnplot
import operator
import distiller
import distiller.apputils as apputils
from distiller.data_loggers import PythonLogger, TensorBoardLogger

import sys
sys.path.insert(0, 'models/')
sys.path.insert(1, 'distiller/')
sys.path.insert(2, 'datasets/')

from cats_and_dogs import *

mod = importlib.import_module("cat-dog_net")

import ai8x
%matplotlib inline

# Logger handle
msglogger = None


## Define Training Configurations (args)

In [92]:
dataset_name = "cats_and_dogs"
dataset_fn = cats_and_dogs_get_datasets
num_classes = 2
model_name = "catdognet"
dimensions = (3,128,128)
workers = 4
batch_size = 32
validation_split = 0.1
log_prefix = "train_log"
log_dir = "jupyter_logging"
data_path = "../Datasets/cats_and_dogs/"
deterministic = True
print_freq = 100
labels = ('dog', 'cat')
lr = 0.001

## Define Helper Functions

In [93]:
class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

## Set up the logger

In [94]:
msglogger = apputils.config_pylogger('logging.conf', log_prefix,
                                        log_dir)

# Log various details about the execution environment.  It is sometimes useful
# to refer to past experiment executions and this information may be useful.
apputils.log_execution_env_state(None, msglogger.logdir)
msglogger.debug("Distiller: %s", distiller.__version__)


pylogger = PythonLogger(msglogger, log_1d=True)
all_loggers = [pylogger]

# tensorboard
tflogger = TensorBoardLogger(msglogger.logdir, log_1d=True, comment='_'+dataset_name)

tflogger.tblogger.writer.add_text('Command line', "args ---")

msglogger.info('dataset_name:%s\ndataset_fn=%s\nnum_classes=%d\nmodel_name=%s\ndimensions=%s\nbatch_size=%d\nvalidation_split=%s\nlr=%f',
                dataset_name,dataset_fn,num_classes,model_name,dimensions,batch_size,validation_split,lr)

Log file for this run: /home/geffencooper/Model_Development/ai8x-training/jupyter_logging/train_log___2022.06.21-174615/train_log___2022.06.21-174615.log
dataset_name:cats_and_dogs
dataset_fn=<function cats_and_dogs_get_datasets at 0x7fcef0358820>
num_classes=2
model_name=catdognet
dimensions=(3, 128, 128)
batch_size=32
validation_split=0.1
lr=0.001000



--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'



## Create and Load the Datasets

In [95]:
args = Args(act_mode_8bit=False)
train_set, test_set = cats_and_dogs_get_datasets((data_path, args), load_train=True, load_test=True)

{'dogs': 0, 'cats': 1}
{'dogs': 0, 'cats': 1}


## Visualize a batch of training data

In [96]:
#train_set.visualize_batch()

## Create the data loaders

In [97]:
train_loader, val_loader, test_loader, _ = apputils.get_data_loaders(
        dataset_fn, (data_path,args), batch_size,
        workers, validation_split, deterministic,1, 1, 1)
msglogger.info('Dataset sizes:\n\ttraining=%d\n\tvalidation=%d\n\ttest=%d',
                   len(train_loader.sampler), len(val_loader.sampler), len(test_loader.sampler))
msglogger.info('Augmentations:%s',train_loader.dataset.transform)
# train_dataloader = DataLoader(train_set,batch_size=batch_size,shuffle=True)
# test_dataloader = DataLoader(test_set,batch_size=batch_size,shuffle=True)

Dataset sizes:
	training=18000
	validation=2000
	test=5000
Augmentations:Compose(
    Resize(size=(128, 128), interpolation=bilinear)
    RandomHorizontalFlip(p=0.5)
    GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 5))
    ToTensor()
)


{'dogs': 0, 'cats': 1}
{'dogs': 0, 'cats': 1}


## Set up the device, cuda or cpu

In [98]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


## Set up the model for training

In [99]:
ai8x.set_device(device=85, simulate=False, round_avg=False)

model = mod.CatsAndDogsClassifier()
        
model = model.to(device)

print(f'Number of Model Params: {count_params(model)}')

# configure tensorboard
dummy_input = torch.randn((1, ) + dimensions)
tflogger.tblogger.writer.add_graph(model.to('cpu'), (dummy_input, ), False)

all_loggers.append(tflogger)
all_tbloggers = [tflogger]

Configuring device: MAX78000, simulate=False.
Number of Model Params: 279562


## Set up the training parameters

In [100]:
num_epochs = 40
msglogger.info('epochs: %d',num_epochs)
optimizer = optim.Adam(model.parameters(), lr=lr)
msglogger.info('Optimizer Type: %s', type(optimizer))
ms_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 25], gamma=0.5)
msglogger.info("lr_schedule:%s","base: "+str(ms_lr_scheduler.base_lrs)+" milestones: "+str(ms_lr_scheduler.milestones)+ " gamma: "+str(ms_lr_scheduler.gamma))
criterion = torch.nn.CrossEntropyLoss().to(device)

qat_policy = {'start_epoch':5,
              'weight_bits':8}
msglogger.info('qat policy: %s',qat_policy)
compression_scheduler = distiller.CompressionScheduler(model)

epochs: 40
Optimizer Type: <class 'torch.optim.adam.Adam'>
lr_schedule:base: [0.001] milestones: Counter({5: 1, 25: 1}) gamma: 0.5
qat policy: {'start_epoch': 5, 'weight_bits': 8}


## Validation Function

In [101]:
def validate(data_loader, model, criterion, loggers, epoch=-1, tflogger=None):
    """Execute the validation/test loop."""

    # store loss stats
    losses = {'objective_loss': tnt.AverageValueMeter()}
    classerr = tnt.ClassErrorMeter(accuracy=True, topk=(1, min(num_classes, 5)))

    # validation set info
    batch_time = tnt.AverageValueMeter()
    total_samples = len(data_loader.sampler)
    batch_size = data_loader.batch_size
    confusion = tnt.ConfusionMeter(num_classes)
    total_steps = (total_samples + batch_size - 1) // batch_size
    msglogger.info('%d samples (%d per mini-batch)', total_samples, batch_size)

    # Switch to evaluation mode
    model.eval()

    end = time.time()
    class_probs = []
    class_preds = []

    # iterate over the batches in the validation set
    for validation_step, (inputs, target) in enumerate(data_loader):
        with torch.no_grad():
            inputs, target = inputs.to(device), target.to(device)
            # compute output from model
            output = model(inputs)
            # correct output for accurate loss calculation
            if args.act_mode_8bit:
                output /= 128.
                for key in model.__dict__['_modules'].keys():
                    if (hasattr(model.__dict__['_modules'][key], 'wide')
                            and model.__dict__['_modules'][key].wide):
                        output /= 256.
            # compute loss
            loss = criterion(output, target)
            # measure accuracy and record loss
            losses['objective_loss'].add(loss.item())
            if len(output.data.shape) <= 2:
                classerr.add(output.data, target)
            else:
                classerr.add(output.data.permute(0, 2, 3, 1).flatten(start_dim=0, end_dim=2),
                                target.flatten())
            
            confusion.add(output.data, target)

            # measure elapsed time
            batch_time.add(time.time() - end)
            end = time.time()

            # store prediction stats
            steps_completed = (validation_step+1)
            if steps_completed % print_freq == 0 or steps_completed == total_steps:
                class_probs_batch = [torch.nn.functional.softmax(el, dim=0) for el in output]
                _, class_preds_batch = torch.max(output, 1)
                class_probs.append(class_probs_batch)
                class_preds.append(class_preds_batch)

                stats = (
                    '',
                    OrderedDict([('Loss', losses['objective_loss'].mean),
                                    ('Top1', classerr.value(1))])
                )
                if num_classes > 5:
                    stats[1]['Top5'] = classerr.value(5)

                distiller.log_training_progress(stats, None, epoch, steps_completed,
                                                total_steps, print_freq, loggers)

                # if tflogger is not None:
                #     test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
                #     test_preds = torch.cat(class_preds)
                #     for i in range(num_classes):
                #         tb_preds = test_preds == i
                #         tb_probs = test_probs[:, i]
                #         tflogger.tblogger.writer.add_pr_curve(str(args.labels[i]), tb_preds,
                #                                             tb_probs, global_step=epoch)

                # if steps_completed == total_steps and tflogger is not None:
                #     def select_n_random(data, labels, features, n=100):
                #         """Selects n random datapoints, their corresponding labels and features"""
                #         assert len(data) == len(labels) == len(features)

                #         perm = torch.randperm(len(data))
                #         return data[perm][:n], labels[perm][:n], features[perm][:n]

                #     # Select up to 100 random images and their target indices
                #     images, labels, features = select_n_random(inputs, target, output,
                #                                                n=min(100, len(inputs)))

                #     # Get the class labels for each image
                #     class_labels = [args.labels[lab] for lab in labels]

                #     tflogger.tblogger.writer.add_embedding(
                #         features,
                #         metadata=class_labels,
                #         label_img=args.visualize_fn(images, args),
                #         global_step=epoch,
                #         tag='verification/embedding'
                #     )

    if num_classes > 5:
        msglogger.info('==> Top1: %.3f    Top5: %.3f    Loss: %.3f\n',
                        classerr.value()[0], classerr.value()[1],
                        losses['objective_loss'].mean)
    else:
        msglogger.info('==> Top1: %.3f    Loss: %.3f\n',
                        classerr.value()[0], losses['objective_loss'].mean)

    msglogger.info('==> Confusion:\n%s\n', str(confusion.value()))
    if tflogger is not None:
        cf = nnplot.confusion_matrix(confusion.value(), labels)
        tflogger.tblogger.writer.add_image('Validation/ConfusionMatrix', cf, epoch,
                                            dataformats='HWC')
    return classerr.value(1), classerr.value(min(num_classes, 5)), losses['objective_loss'].mean

## Run the trianing loop

In [102]:
# keep track of incorrect predictions
wrong_samples = None
wrong_preds = None
actual_preds = None

# store model history across epochs
perf_scores_history = []
model = model.to(device)

name = model_name

# training loop
for epoch in range(0, num_epochs):
    # check if need to switch to QAT
    if epoch > 0 and epoch == qat_policy['start_epoch']:
        print('QAT is starting!')
        # Fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
        ai8x.fuse_bn_layers(model)

        # Switch model from unquantized to quantized for QAT
        ai8x.initiate_qat(model, qat_policy)

        # Model is re-transferred to GPU in case parameters were added
        model.to(device)

        # Empty the performance scores list for QAT operation
        perf_scores_history = []
        name = f'{model_name}_qat'

    # store loss and training stats
    losses = {'objective_loss': tnt.AverageValueMeter()}
    classerr = tnt.ClassErrorMeter(accuracy=True, topk=(1, min(num_classes, 5)))
    batch_time = tnt.AverageValueMeter()
    data_time = tnt.AverageValueMeter()

    # logging stats
    total_samples = len(train_loader.sampler)
    batch_size = train_loader.batch_size
    steps_per_epoch = (total_samples + batch_size - 1) // batch_size
    msglogger.info('Training epoch: %d samples (%d per mini-batch)', total_samples, batch_size)

    # Switch to train mode
    model.train()
    acc_stats = []
    end = time.time()

    # iterate over all batches in the dataset
    for train_step, (inputs, target) in enumerate(train_loader):
        # Measure data loading time
        data_time.add(time.time() - end)
        inputs, target = inputs.to(device), target.to(device)

        # forward pass and loss calculation
        output = model(inputs)
        loss = criterion(output, target)

        # on the last batch store the stats for the epoch
        if train_step >= len(train_loader)-2:
            if len(output.data.shape) <= 2:
                classerr.add(output.data, target)
            else:
                classerr.add(output.data.permute(0, 2, 3, 1).flatten(start_dim=0, end_dim=2),
                                target.flatten())
            acc_stats.append([classerr.value(1), classerr.value(min(num_classes, 5))])

        # add the loss for each batch
        losses["objective_loss"].add(loss.item())

        # reset the optimizer
        optimizer.zero_grad()

        # backwards pass and parameter update
        loss.backward()
        optimizer.step()

        # track batch stats
        batch_time.add(time.time() - end)
        steps_completed = (train_step+1)

        # log stats every 10 batches
        if steps_completed % print_freq == 0 or steps_completed == steps_per_epoch:
            # Log some statistics
            errs = OrderedDict()
            if classerr.n != 0:
                errs['Top1'] = classerr.value(1)
                if num_classes > 5:
                    errs['Top5'] = classerr.value(5)
            else:
                errs['Top1'] = None
                errs['Top5'] = None

            stats_dict = OrderedDict()
            for loss_name, meter in losses.items():
                stats_dict[loss_name] = meter.mean
            stats_dict.update(errs)
            
            stats_dict['LR'] = optimizer.param_groups[0]['lr']
            stats_dict['Time'] = batch_time.mean
            stats = ('Performance/Training/', stats_dict)
            params = None
            distiller.log_training_progress(stats,
                                            params,
                                            epoch, steps_completed,
                                            steps_per_epoch, print_freq,
                                            all_loggers)
        end = time.time()



    # after a training epoch, do validation
    msglogger.info('--- validate (epoch=%d)-----------', epoch)
    top1, top5, vloss = validate(val_loader, model, criterion, [pylogger], epoch, tflogger)

    # store validation stats
    stats = ('Performance/Validation/', OrderedDict([('Loss', vloss), ('Top1', top1)]))
    if num_classes > 5:
        stats[1]['Top5'] = top5

    distiller.log_training_progress(stats, None, epoch, steps_completed=0, total_steps=1,
                                            log_freq=1, loggers=all_tbloggers)

    perf_scores_history.append(distiller.MutableNamedTuple({'top1': top1, 'top5': top5,
                                                            'epoch': epoch}))
    # Keep perf_scores_history sorted from best to worst
    # Sort by top1 as main sort key, then sort by top5 and epoch
    perf_scores_history.sort(key=operator.attrgetter('top1', 'top5', 'epoch'),reverse=True)
    for score in perf_scores_history[:1]:
        if num_classes > 5:
            msglogger.info('==> Best [Top1: %.3f   Top5: %.3f  on epoch: %d]',
                            score.top1, score.top5,score.epoch)
        else:
            msglogger.info('==> Best [Top1: %.3f on epoch: %d]',
                            score.top1, score.epoch)

    # Save the checkpoint
    is_best = epoch == perf_scores_history[0].epoch
    checkpoint_extras = {'current_top1': top1,
                        'best_top1': perf_scores_history[0].top1,
                        'best_epoch': perf_scores_history[0].epoch}

    apputils.save_checkpoint(epoch, model_name, model, optimizer=optimizer,
                                scheduler=compression_scheduler, extras=checkpoint_extras,
                                is_best=is_best, name=name,
                                dir=msglogger.logdir)

    ms_lr_scheduler.step()

# Finally run results on the test set
top1, top5, losses = validate(test_loader, model, criterion, [pylogger],epoch,None)
msglogger.info('==> Best [Top1: %.3f   Top5: %.3f  on test set]', top1, top5)



#     running_loss = []
#     train_start = time.time()
#     model.train()
#     for idx, (image, label) in enumerate(train_dataloader):
#         image = image.to(device)
#         label = label.type(torch.long).to(device)
#         optimizer.zero_grad()
        
#         model_out = model(image)
        
#         loss = criterion(model_out, label)
#         loss.backward()
#         optimizer.step()
        
#         running_loss.append(loss.cpu().detach().numpy())

#     mean_loss = np.mean(running_loss)
#     train_end = time.time()
#     print("Epoch: {}/{}\t LR: {}\t Train Loss: {:.4f}\t Dur: {:.2f} sec.".format(epoch+1, num_epochs, ms_lr_scheduler.get_lr(), mean_loss, (train_end-train_start)))
    
#     model.eval()
#     acc = 0.
#     acc_weight = 0
#     with torch.no_grad():
#         for image, label in test_dataloader:
#             image = image.to(device)
#             label = label.type(torch.long).to(device)
#             model_out = model(image)
#             label_out = torch.argmax(model_out, dim=1)

#             # display wrong outputs
#             pred = model_out.argmax(dim=1, keepdim=True) # get the idxs of the max output
#             wrong_idx = (pred != label.view_as(pred)).nonzero()[:, 0] # get wrong predictions
#             wrong_samples = image[wrong_idx]
#             wrong_preds = pred[wrong_idx]
#             actual_preds = label.view_as(pred)[wrong_idx]
            
#             # test_set.viz_mispredict(wrong_samples,wrong_preds,actual_preds)
            
#             tp = torch.sum(label_out == label)
#             acc_batch = (tp / label_out.numel()).detach().item()
#             acc += label_out.shape[0] * acc_batch
#             acc_weight += label_out.shape[0]
            
#         total_acc = 100 * (acc / acc_weight)
#         if epoch == qat_policy['start_epoch']: best_acc = 0
#         if total_acc > best_acc:
#             best_acc = total_acc
#             checkpoint_extras = {'current_top1': best_acc,
#                                  'best_top1': best_acc,
#                                  'best_epoch': epoch}
#             model_name = 'catdognet'
#             model_prefix = f'{model_name}' if epoch < qat_policy['start_epoch'] else (f'qat_{model_name}')
#             apputils.save_checkpoint(epoch, model_name, model, optimizer=optimizer,
#                                      scheduler=None, extras=checkpoint_extras,
#                                      is_best=True, name=model_prefix,
#                                      dir='.')
#             print(f'Best model saved with accuracy: {best_acc:.2f}%')
            
#         print('\t\t Test Acc: {:.2f}'.format(total_acc))
        
#     ms_lr_scheduler.step()
# #test_set.visualize_batch(model,device)
# test_set.viz_mispredict(wrong_samples,wrong_preds,actual_preds)


Training epoch: 18000 samples (32 per mini-batch)
/home/geffencooper/Model_Development/ai8x-training/venv/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "

Epoch: [0][  100/  563]    objective_loss 0.692981                                        LR 0.000250    Time 0.023976    
Epoch: [0][  200/  563]    objective_loss 0.690697                                        LR 0.000250    Time 0.022691    
Epoch: [0][  300/  563]    objective_loss 0.689208                                        LR 0.000250    Time 0

KeyboardInterrupt: 

## Quantize the model

You must change the kernel to execute within the ai8x-synthesis virtual environment

In [1]:
%load_ext autoreload
%autoreload 2
%run ../ai8x-synthesis/quantize.py jupyter_logging/train_log___2022.06.21-165010/cat-dog_net_qat_best.pth.tar jupyter_logging/train_log___2022.06.21-165010/cat-dog_net_qat_best-q.pth.tar --device MAX78000 -v

Configuring device: MAX78000
Converting checkpoint file jupyter_logging/train_log___2022.06.21-165010/cat-dog_net_qat_best.pth.tar to jupyter_logging/train_log___2022.06.21-165010/cat-dog_net_qat_best-q.pth.tar

Model keys (state_dict):
conv1.output_shift, conv1.weight_bits, conv1.bias_bits, conv1.quantize_activation, conv1.adjust_output_shift, conv1.shift_quantile, conv1.op.weight, conv2.output_shift, conv2.weight_bits, conv2.bias_bits, conv2.quantize_activation, conv2.adjust_output_shift, conv2.shift_quantile, conv2.op.weight, conv3.output_shift, conv3.weight_bits, conv3.bias_bits, conv3.quantize_activation, conv3.adjust_output_shift, conv3.shift_quantile, conv3.op.weight, conv4.output_shift, conv4.weight_bits, conv4.bias_bits, conv4.quantize_activation, conv4.adjust_output_shift, conv4.shift_quantile, conv4.op.weight, conv4.op.bias, conv5.output_shift, conv5.weight_bits, conv5.bias_bits, conv5.quantize_activation, conv5.adjust_output_shift, conv5.shift_quantile, conv5.op.weight, con

## Evaluate Quantized Model

Change virtual environment back to ai8x-training and rerun the first cell with the imports

In [1]:
import importlib

import os
from matplotlib.image import imread
import numpy as np
import matplotlib.pyplot as plt
import time
import logging
from collections import OrderedDict

try:
    import tensorboard  # pylint: disable=import-error
    import tensorflow  # pylint: disable=import-error
    tensorflow.io.gfile = tensorboard.compat.tensorflow_stub.io.gfile
except (ModuleNotFoundError, AttributeError):
    pass

import torch
from torch import nn
import torchnet.meter as tnt
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import nnplot
import operator
import distiller
import distiller.apputils as apputils
from distiller.data_loggers import PythonLogger, TensorBoardLogger

import sys
sys.path.insert(0, 'models/')
sys.path.insert(1, 'distiller/')
sys.path.insert(2, 'datasets/')

from cats_and_dogs import *

mod = importlib.import_module("cat-dog_net")

import ai8x
%matplotlib inline

# Logger handle
msglogger = None

class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
load_model_path = 'jupyter_logging/train_log___2022.06.21-165010/cat-dog_net_qat_best-q.pth.tar'
# Change this path to match file system layout
data_path = "../Datasets/cats_and_dogs/"

ai8x.set_device(device=85, simulate=True, round_avg=False)

model = mod.CatsAndDogsClassifier()
                          
checkpoint = torch.load(load_model_path, map_location=lambda storage, loc: storage)
ai8x.fuse_bn_layers(model)
model = apputils.load_lean_checkpoint(model, load_model_path, model_device=device)
ai8x.update_model(model)
model = model.to(device)

Configuring device: MAX78000, simulate=True.


In [3]:
args = Args(act_mode_8bit=True)

_, test_set = train_set, test_set = cats_and_dogs_get_datasets((data_path, args), load_train=True, load_test=True)
test_dataloader = DataLoader(test_set, batch_size=128, shuffle=False)

{'dogs': 0, 'cats': 1}
{'dogs': 0, 'cats': 1}


In [4]:
model.eval()
acc = 0.
acc_weight = 0
with torch.no_grad():
    for image, label in test_dataloader:
        image = image.to(device)
        label = label.type(torch.long).to(device)
        model_out = model(image)
        label_out = torch.argmax(model_out, dim=1)

        tp = torch.sum(label_out == label)
        acc_batch = (tp / label_out.numel()).detach().item()
        acc += label_out.shape[0] * acc_batch
        acc_weight += label_out.shape[0]

    total_acc = 100 * (acc / acc_weight)
print(f'Quantized accuracy: {total_acc:.2f}%')

Quantized accuracy: 89.00%


In [1]:
for image, label in test_dataloader:
    break

im_sample = (image[0].detach().cpu().numpy()).astype(np.int64)

np.save('sample_cats_and_dogs.npy', im_sample)

NameError: name 'test_dataloader' is not defined

## Model Synthesis

In [4]:
%cd ../ai8x-synthesis/
%run ai8xize.py --verbose --log --test-dir sdk/Examples/MAX78000/CNN --prefix cats_and_dogs --checkpoint-file ../ai8x-training/jupyter_logging/train_log___2022.06.21-165010/cat-dog_net_qat_best-q.pth.tar --config-file networks/cats_and_dogs.yaml --device MAX78000 --softmax --compact-data --mexpress --timer 0 --fifo --display-checkpoint

Storing weights...   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━  84%

Output()

WARNING:

Layer 1: All output values for the given sample input are zero. The generated known-answer test for this network may not be meaningful. See the log file for details.


WARNING: